In [1]:
# Fix randomness and hide warnings
seed = 42
%load_ext autoreload
%autoreload 2

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ['MPLCONFIGDIR'] = os.getcwd()+'/configs/'

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)

import numpy as np
np.random.seed(seed)

import logging

import random
random.seed(seed)

# Fix randomness and hide warnings
seed = 42

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ['MPLCONFIGDIR'] = os.getcwd()+'/configs/'

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)

import numpy as np
np.random.seed(seed)

import logging

import random
random.seed(seed)

# Import tensorflow
import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl
tf.autograph.set_verbosity(0)
tf.get_logger().setLevel(logging.ERROR)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)
print(tf.__version__)

import matplotlib.pyplot as plt
plt.rc('font', size=16)

from C2.notebooks.Utils import split_dataset, build_sequences_optimized
from C2.notebooks.data_augmentation import augment_data

2.15.0


In [2]:
input_folder = '../data/training_dataset/'
training_data = np.load(input_folder + 'training_data.npy')
valid_periods = np.load(input_folder + 'valid_periods.npy')
categories = np.load(input_folder + 'categories.npy')

In [3]:

dataset, labels = build_sequences_optimized(training_data, valid_periods)
train_validation_data, train_validation_labels, test_data, test_labels = split_dataset(dataset, labels)
train_data, train_labels, validation_data, validation_labels = split_dataset(train_validation_data, train_validation_labels, augment_train_data=True, num_augmentations=3)

train_data = train_data.reshape((train_data.shape[0], train_data.shape[1], 1))
train_labels = train_labels.reshape((train_labels.shape[0], train_labels.shape[1], 1))
test_data = test_data.reshape((test_data.shape[0], test_data.shape[1], 1))
test_labels = test_labels.reshape((test_labels.shape[0], test_labels.shape[1], 1))
test_data = test_data.reshape((test_data.shape[0], test_data.shape[1], 1))
test_labels = test_labels.reshape((test_labels.shape[0], test_labels.shape[1], 1))
train_data.shape

Dataset shape:  (48000, 200)
Train data shape:  (38400, 200)
Test data shape:  (9600, 200)
Augmenting training data
Augmentation round: 0
Augmentation round: 1
Augmentation round: 2
Train data shape:  (276480, 200)
Test data shape:  (7680, 200)


(276480, 200, 1)

In [2]:
def build_LSTM_ENC_DEC_model_params(input_shape, output_shape, learning_rate=0.001, num_layers=2, num_units_lstm=32, num_units_final_dense = 128, dropout=0.2):
    assert input_shape[0] >= output_shape[0], "Input time steps should be >= output time steps"

    # Input layer
    input_layer = tf.keras.layers.Input(shape=input_shape, name='input_layer')

    # LSTM layers
    x = input_layer
    for _ in range(num_layers):
        x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(num_units_lstm, name='lstm', return_sequences=True))(x)
        x = tf.keras.layers.Dropout(dropout)(x)
    x = tf.keras.layers.Flatten()(x)
    flatten_input = tf.keras.layers.Flatten()(input_layer)
    # Skipper layer
    x = tf.keras.layers.Concatenate()([x, flatten_input])
    x = tf.keras.layers.Dense(num_units_final_dense, activation='relu')(x)
    # Output layer
    output_layer = tf.keras.layers.Dense(output_shape[0], name='output_layer', activation='sigmoid')(x)
    # Model instantiation
    model = tf.keras.models.Model(inputs=input_layer, outputs=output_layer, name='Better_CONV_LSTM_model')

    # Compiling the model
    model.compile(loss=tf.keras.losses.MeanSquaredError(),
                  metrics=[tf.keras.metrics.MeanAbsoluteError()],
                  optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate))

    return model
    


## Optuna optimization

In [11]:
import optuna as opt
from optuna.integration import TFKerasPruningCallback
from optuna.pruners import MedianPruner
from optuna.samplers import TPESampler

def objective(trial):
    # Clear clutter from previous session graphs.
    tf.keras.backend.clear_session()

    # Hyperparameters to be tuned by Optuna.
    trial.suggest_float("learning_rate", 1e-5, 1e-1, log=True)
    trial.suggest_int("num_layers", 1, 3)
    trial.suggest_int("num_units_lstm", 16, 128, log=True)
    trial.suggest_int("num_units_final_dense", 16, 128, log=True)
    trial.suggest_float("dropout", 0.0, 0.5)
    
    batch_size = 256
    epochs = 30

    # Build the model.
    model = build_LSTM_ENC_DEC_model_params(input_shape, output_shape, **trial.params)

    # Train the model.
    model.fit(
        x = train_data,
        y = train_labels,
        batch_size = batch_size,
        epochs = epochs,
        validation_data=(validation_data, validation_labels),
        callbacks = [
            tfk.callbacks.EarlyStopping(monitor='val_loss', mode='min', min_delta=0.0001, patience=3, restore_best_weights=True),
            tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=2, factor=0.1, min_lr=1e-5),
            TFKerasPruningCallback(trial, "val_loss"),
        ]
    )

    # Evaluate the model accuracy on the validation set.
    score = model.evaluate(validation_data, validation_labels, verbose=0)
    return score[0]

input_shape = train_data.shape[1:]
output_shape = train_labels.shape[1:]

# create a study object and optimize the objective function.
study = opt.create_study(direction="minimize", pruner=MedianPruner(n_warmup_steps=1), sampler=TPESampler(seed=seed), study_name='LSTM_ENC_DEC_model_params', storage='sqlite:///../data/optuna.db', load_if_exists=True)
study.optimize(objective, n_trials=100)

[I 2023-12-15 23:49:57,350] Using an existing study with name 'LSTM_ENC_DEC_model_params' instead of creating a new one.


Epoch 1/30
1080/1080 [==============================] - 174s 155ms/step - loss: 0.0280 - mean_absolute_error: 0.1249 - val_loss: 0.0310 - val_mean_absolute_error: 0.1276 - lr: 3.1489e-04
Epoch 2/30
1080/1080 [==============================] - 167s 155ms/step - loss: 0.0207 - mean_absolute_error: 0.1058 - val_loss: 0.0291 - val_mean_absolute_error: 0.1212 - lr: 3.1489e-04
Epoch 3/30
1080/1080 [==============================] - 167s 155ms/step - loss: 0.0191 - mean_absolute_error: 0.1013 - val_loss: 0.0277 - val_mean_absolute_error: 0.1175 - lr: 3.1489e-04
Epoch 4/30
1080/1080 [==============================] - 167s 155ms/step - loss: 0.0181 - mean_absolute_error: 0.0983 - val_loss: 0.0268 - val_mean_absolute_error: 0.1151 - lr: 3.1489e-04
Epoch 5/30
1080/1080 [==============================] - 168s 155ms/step - loss: 0.0173 - mean_absolute_error: 0.0959 - val_loss: 0.0272 - val_mean_absolute_error: 0.1161 - lr: 3.1489e-04
Epoch 6/30
1080/1080 [==============================] - 168s 156m

[I 2023-12-16 00:09:46,858] Trial 5 finished with value: 0.026806823909282684 and parameters: {'learning_rate': 0.00031489116479568613, 'num_layers': 3, 'num_units_lstm': 73, 'num_units_final_dense': 55, 'dropout': 0.07800932022121826}. Best is trial 4 with value: 0.026405544951558113.


Epoch 1/30
1080/1080 [==============================] - 64s 56ms/step - loss: 0.0411 - mean_absolute_error: 0.1570 - val_loss: 0.0339 - val_mean_absolute_error: 0.1374 - lr: 4.2071e-05
Epoch 2/30
1080/1080 [==============================] - 60s 56ms/step - loss: 0.0305 - mean_absolute_error: 0.1313 - val_loss: 0.0324 - val_mean_absolute_error: 0.1322 - lr: 4.2071e-05
Epoch 3/30
1080/1080 [==============================] - 61s 56ms/step - loss: 0.0269 - mean_absolute_error: 0.1224 - val_loss: 0.0306 - val_mean_absolute_error: 0.1271 - lr: 4.2071e-05
Epoch 4/30
1080/1080 [==============================] - 61s 56ms/step - loss: 0.0247 - mean_absolute_error: 0.1168 - val_loss: 0.0300 - val_mean_absolute_error: 0.1251 - lr: 4.2071e-05
Epoch 5/30
1080/1080 [==============================] - 61s 56ms/step - loss: 0.0231 - mean_absolute_error: 0.1126 - val_loss: 0.0303 - val_mean_absolute_error: 0.1249 - lr: 4.2071e-05
Epoch 6/30
1080/1080 [==============================] - 61s 56ms/step - los

[I 2023-12-16 00:25:04,428] Trial 6 finished with value: 0.028541535139083862 and parameters: {'learning_rate': 4.207053950287933e-05, 'num_layers': 1, 'num_units_lstm': 97, 'num_units_final_dense': 55, 'dropout': 0.35403628889802274}. Best is trial 4 with value: 0.026405544951558113.


...

Epoch 1/30
1080/1080 [==============================] - 50s 43ms/step - loss: 0.0276 - mean_absolute_error: 0.1235 - val_loss: 0.0298 - val_mean_absolute_error: 0.1248 - lr: 0.0019
Epoch 2/30
1080/1080 [==============================] - 42s 39ms/step - loss: 0.0208 - mean_absolute_error: 0.1063 - val_loss: 0.0285 - val_mean_absolute_error: 0.1191 - lr: 0.0019
Epoch 3/30
1079/1080 [============================>.] - ETA: 0s - loss: 0.0196 - mean_absolute_error: 0.1027

[I 2023-12-16 01:27:19,023] Trial 16 pruned. Trial was pruned at epoch 2.


Epoch 1/30
1080/1080 [==============================] - 39s 32ms/step - loss: 0.0237 - mean_absolute_error: 0.1139 - val_loss: 0.0268 - val_mean_absolute_error: 0.1167 - lr: 0.0043
Epoch 2/30
 113/1080 [==>...........................] - ETA: 28s - loss: 0.0194 - mean_absolute_error: 0.1025

[W 2023-12-16 01:28:03,738] Trial 17 failed with parameters: {'learning_rate': 0.004279913261876128, 'num_layers': 2, 'num_units_lstm': 16, 'num_units_final_dense': 88, 'dropout': 0.4812801742938247} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/home/federico/Documents/projects/rec-sys/RecSys-2023-polimi/venv/lib/python3.10/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_39760/3123814481.py", line 24, in objective
    model.fit(
  File "/home/federico/Documents/projects/rec-sys/RecSys-2023-polimi/venv/lib/python3.10/site-packages/keras/src/utils/traceback_utils.py", line 65, in error_handler
    return fn(*args, **kwargs)
  File "/home/federico/Documents/projects/rec-sys/RecSys-2023-polimi/venv/lib/python3.10/site-packages/keras/src/engine/training.py", line 1807, in fit
    tmp_logs = self.train_function(iterator)
  File "/home/federico/Documents/projects/rec-s

KeyboardInterrupt: 

In [12]:
study.best_params

{'learning_rate': 0.002555085039485097,
 'num_layers': 2,
 'num_units_lstm': 28,
 'num_units_final_dense': 31,
 'dropout': 0.3218563683119118}

In [19]:

input_shape = train_data.shape[1:]
output_shape = train_labels.shape[1:]
batch_size = 256
epochs = 10

model = build_LSTM_ENC_DEC_model_params(input_shape, output_shape, **study.best_params)
model.summary()
tfk.utils.plot_model(model, expand_nested=True, show_shapes=True) 

Model: "Better_CONV_LSTM_model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_layer (InputLayer)    [(None, 200, 1)]             0         []                            
                                                                                                  
 bidirectional_6 (Bidirecti  (None, 64)                   8704      ['input_layer[0][0]']         
 onal)                                                                                            
                                                                                                  
 dropout_6 (Dropout)         (None, 64)                   0         ['bidirectional_6[0][0]']     
                                                                                                  
 repeat_vector_3 (RepeatVec  (None, 18, 64)               0         ['dropout

In [20]:
# Train the model
history = model.fit(
    x = train_data,
    y = augmented_train_labels,
    batch_size = batch_size,
    epochs = epochs,
    # validation_data=(validation_data, validation_labels),
    callbacks = [
        tfk.callbacks.EarlyStopping(monitor='val_loss', mode='min', min_delta=0.0003, patience=12, restore_best_weights=True),
        tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=8, factor=0.1, min_lr=1e-5)
    ]
).history

Epoch 1/10
1688/1688 [==============================] - 41s 20ms/step - loss: 0.0231 - mean_absolute_error: 0.1097 - lr: 0.0010
Epoch 2/10
1688/1688 [==============================] - 34s 20ms/step - loss: 0.0204 - mean_absolute_error: 0.1019 - lr: 0.0010
Epoch 3/10
1688/1688 [==============================] - 35s 21ms/step - loss: 0.0196 - mean_absolute_error: 0.0995 - lr: 0.0010
Epoch 4/10
1688/1688 [==============================] - 35s 21ms/step - loss: 0.0191 - mean_absolute_error: 0.0981 - lr: 0.0010
Epoch 5/10
1688/1688 [==============================] - 35s 21ms/step - loss: 0.0186 - mean_absolute_error: 0.0967 - lr: 0.0010
Epoch 6/10
1688/1688 [==============================] - 35s 21ms/step - loss: 0.0182 - mean_absolute_error: 0.0955 - lr: 0.0010
Epoch 7/10
1688/1688 [==============================] - 35s 21ms/step - loss: 0.0177 - mean_absolute_error: 0.0943 - lr: 0.0010
Epoch 8/10
1688/1688 [==============================] - 35s 21ms/step - loss: 0.0174 - mean_absolute_err

In [18]:
from C2.notebooks import Utils
from importlib import reload
reload(Utils)
test_data_predictions = model.predict(test_data)
Utils.evaluate_model(test_data_predictions, test_labels)

300/300 [==============================] - 4s 7ms/step
MAE: 0.11244506225454622
MSE: 0.026357403430591904
RMSE: 0.16234963329367857


In [ ]:
# Evaluate random samples
Utils.evaluate_model(np.random.rand(*test_data_predictions.shape), test_labels)

In [ ]:
for _ in range(10):
    idx = np.random.randint(0, len(test_data))
    Utils.plot_predictions(test_data, test_data_predictions, test_labels,idx)
    # Utils.plot_predictions(train_data, test_data_predictions, train_labels,idx)

# Retraining for submission

In [3]:
input_folder = '../data/training_dataset/'
training_data = np.load(input_folder + 'training_data.npy')
valid_periods = np.load(input_folder + 'valid_periods.npy')
categories = np.load(input_folder + 'categories.npy')

dataset, labels = build_sequences_optimized(training_data, valid_periods)
train_data, train_labels = augment_data(dataset, labels)
train_data = train_data.reshape((train_data.shape[0], train_data.shape[1], 1))
train_labels = train_labels.reshape((train_labels.shape[0], train_labels.shape[1], 1))

input_shape = train_data.shape[1:]
output_shape = train_labels.shape[1:]
batch_size = 256
epochs = 10

params = {'learning_rate': 0.002555085039485097,
 'num_layers': 2,
 'num_units_lstm': 28,
 'num_units_final_dense': 31,
 'dropout': 0.3218563683119118}
model = build_LSTM_ENC_DEC_model_params(input_shape, output_shape, **params)

# Train the model
history = model.fit(
    x = train_data,
    y = train_labels,
    batch_size = batch_size,
    epochs = epochs,
).history

model.save('../data/models/LSTM_ENC_DEC_model_params.h5')

Dataset shape:  (48000, 200)
Augmentation round: 0
Augmentation round: 1
Augmentation round: 2
Epoch 1/10


I0000 00:00:1702686771.541976   81147 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1688/1688 [==============================] - 80s 44ms/step - loss: 0.0265 - mean_absolute_error: 0.1199
Epoch 2/10
1688/1688 [==============================] - 74s 44ms/step - loss: 0.0216 - mean_absolute_error: 0.1072
Epoch 3/10
1688/1688 [==============================] - 74s 44ms/step - loss: 0.0201 - mean_absolute_error: 0.1032
Epoch 4/10
1688/1688 [==============================] - 74s 44ms/step - loss: 0.0192 - mean_absolute_error: 0.1008
Epoch 5/10
1688/1688 [==============================] - 74s 44ms/step - loss: 0.0184 - mean_absolute_error: 0.0987
Epoch 6/10
1688/1688 [==============================] - 74s 44ms/step - loss: 0.0178 - mean_absolute_error: 0.0969
Epoch 7/10
1688/1688 [==============================] - 74s 44ms/step - loss: 0.0173 - mean_absolute_error: 0.0957
Epoch 8/10
1688/1688 [==============================] - 75s 44ms/step - loss: 0.0169 - mean_absolute_error: 0.0946
Epoch 9/10
1688/1688 [==============================] - 74s 44ms/step - loss: 0.0166 - mean